In [15]:
from TPNN.tools.CNNarchitecture import *
from TPNN.tools.activation_functions import *
import numpy as np

## Test net creation

In [16]:
# create net:
cnnLayer = CNNlayer((1, 10, 10), sigmoid, 2, (3,3), 2)
poolingLayer = MaxPoolingLayer((2,2), cnnLayer.output_shape)
reformatLayer = ReformatLayer(poolingLayer.output_shape)
denseLayer = DenseLayer(sigmoid, 3, reformatLayer.output_shape[0])
softmaxLayer = SoftmaxLayer(3)

net = Net()
net.add_layer(cnnLayer)
net.add_layer(poolingLayer)
net.add_layer(reformatLayer)
net.add_layer(denseLayer)
net.add_layer(softmaxLayer)
net.print_config()

Layers count: 5
layers configuration: 

CNN layer:
 input shape - (1, 10, 10); output shape - (2, 4, 4)
 cores: 
[0]:
     0.6552353176551609 0.5527611672823729 0.3619040262655975 
     -0.11716042762885048 -0.5171855911427634 -0.4209357585771494 
     -0.7967428086631978 0.20794821560436305 0.3677583455730815 
[1]:
     0.7078379917073885 -0.48398198129754944 0.8639424867662429 
     -0.5667194604851593 -0.7088557503185642 0.09261000535433794 
     -0.31718823982253697 0.03245067377198496 0.078630448925191 
 biases: 
     0.14001375266625282 0.2972643720903607 

MaxPooling layer
 input shape - (2, 4, 4); output shape - (2, 2, 2)
 core shape: (2, 2)

Reformat layer
 input shape - (2, 2, 2); output shape - (4,)

Dense layer
 neuron count: 3
 input shape - (4,); output shape - (3,)
 weight matrix:
     -0.32428719995136035 0.2941424002183488 0.2510167403374175 
     0.7997647241100501 -0.566278605924053 0.6032651599402437 
     -0.7084714652634527 0.9302203305939973 0.05019913765618789 


In [17]:
input_data = np.random.rand(1, 10, 10)

print("input:")
print(input_data)

input:
[[[0.74815122 0.28663094 0.24601699 0.29001152 0.92878492 0.82943108
   0.58088552 0.89576043 0.0097744  0.28268024]
  [0.42399876 0.14722724 0.44795247 0.86797992 0.98522355 0.89485448
   0.8115313  0.05005084 0.83376104 0.83224247]
  [0.28172778 0.27578211 0.51780243 0.57941163 0.8175827  0.24530307
   0.303582   0.19353171 0.80704552 0.34608439]
  [0.88950311 0.32494062 0.75964866 0.27768566 0.9981862  0.86493783
   0.97293713 0.6516111  0.12274085 0.70465252]
  [0.81062052 0.35526114 0.03336966 0.90963862 0.20880974 0.72620271
   0.31351587 0.09547871 0.86163391 0.22521066]
  [0.58170654 0.5046003  0.01426874 0.61602558 0.10142351 0.3162055
   0.95355193 0.9036533  0.0923331  0.26471475]
  [0.24886748 0.2022504  0.27580047 0.63757951 0.9412128  0.77107622
   0.01370575 0.22560414 0.95192664 0.25215577]
  [0.02781353 0.27524145 0.45122446 0.25629265 0.72530835 0.92984676
   0.0680786  0.67382522 0.53729352 0.32270771]
  [0.02743307 0.96206212 0.32702814 0.47857515 0.96228072 

In [18]:
print(net.get_output(input_data))

[0.3084215  0.30503305 0.38654545]


## Test CNN layer output

In [19]:
def print_info(input, net):
    print("input:")
    print(input)
    print("net config")
    net.print_config()

cnnLayer = CNNlayer((2,4,4), ident, 2, (2,2), 2)
cores = np.array([np.eye(2), np.eye(2)])
cnnLayer.set_cores(cores)
cnnLayer.set_biases(np.array([0, 0]))

net = Net()
net.add_layer(cnnLayer)

input = np.array([np.eye(4),np.eye(4)])

print_info(input, net)

input:
[[[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]]

 [[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]]]
net config
Layers count: 1
layers configuration: 

CNN layer:
 input shape - (2, 4, 4); output shape - (2, 2, 2)
 cores: 
[0]:
     1.0 0.0 
     0.0 1.0 
[1]:
     1.0 0.0 
     0.0 1.0 
 biases: 
     0 0 



In [20]:
print(net.get_output(input))

[[[4. 0.]
  [0. 4.]]

 [[4. 0.]
  [0. 4.]]]


## Test pooling layer

In [21]:
net = Net()
poolingLayer = MaxPoolingLayer((2,2), (2,4,4))
net.add_layer(poolingLayer)

input = np.array([[[1,2,1,2], [1,2,1,2], [1,2,1,2], [1,2,1,2]], [[1,2,1,2], [1,2,1,2], [1,2,1,2], [1,2,1,2]]])

print_info(input, net)

input:
[[[1 2 1 2]
  [1 2 1 2]
  [1 2 1 2]
  [1 2 1 2]]

 [[1 2 1 2]
  [1 2 1 2]
  [1 2 1 2]
  [1 2 1 2]]]
net config
Layers count: 1
layers configuration: 

MaxPooling layer
 input shape - (2, 4, 4); output shape - (2, 2, 2)
 core shape: (2, 2)



In [22]:
print(net.get_output(input))

[[[2. 2.]
  [2. 2.]]

 [[2. 2.]
  [2. 2.]]]


## Test dense layer

In [23]:
denseLayer = DenseLayer(ident, 10, 10)
denseLayer.set_weighs(np.eye(10))
denseLayer.set_biases(np.zeros((10, 1)))

net = Net()
net.add_layer(denseLayer)

input = np.arange(0,10)

print_info(input, net)

input:
[0 1 2 3 4 5 6 7 8 9]
net config
Layers count: 1
layers configuration: 

Dense layer
 neuron count: 10
 input shape - (10,); output shape - (10,)
 weight matrix:
     1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
     0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
     0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
     0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 
     0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 
     0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 
     0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 
     0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 
     0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 
     0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 
 biases vector: 
     0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 



In [24]:
net.get_output(input)

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

## Test reformat layer

In [25]:
net = Net()
reformatLayer = ReformatLayer((2, 4, 4))
net.add_layer(reformatLayer)

input = np.ones((2,4,4))

print_info(input, net)

input:
[[[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]

 [[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]]
net config
Layers count: 1
layers configuration: 

Reformat layer
 input shape - (2, 4, 4); output shape - (16,)



In [26]:
net.get_output(input)

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

## Test softmax layer

In [27]:
net = Net()
softmaxLayer = SoftmaxLayer(10)
net.add_layer(softmaxLayer)

input = np.arange(1, 11)
print_info(input, net)

input:
[ 1  2  3  4  5  6  7  8  9 10]
net config
Layers count: 1
layers configuration: 

Softmax layer
 input shape - (10,); output shape - (10,)



In [28]:
output = net.get_output(input)
print(output)
print(np.sum(output))

[7.80134161e-05 2.12062451e-04 5.76445508e-04 1.56694135e-03
 4.25938820e-03 1.15782175e-02 3.14728583e-02 8.55520989e-02
 2.32554716e-01 6.32149258e-01]
1.0
